In [1]:
import arkouda as ak
import arachne as ar
import numpy as np
from scipy import stats

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
ak.connect("n117", 5555)

connected to arkouda server tcp://*:5555


In [3]:
def test_watts_strogatz():
    """Test cases for Watts-Strogatz graph generation"""
    test_cases = [
        {"n": 100, "k": 4, "p": 0.1},
        {"n": 500, "k": 6, "p": 0.05},
        {"n": 1000, "k": 8, "p": 0.2},
        {"n": 2000, "k": 10, "p": 0.15},
        {"n": 5000, "k": 12, "p": 0.25},
        {"n": 200, "k": 6, "p": 0.3},
        {"n": 300, "k": 8, "p": 0.35},
        {"n": 400, "k": 10, "p": 0.4},
        {"n": 800, "k": 12, "p": 0.45},
        {"n": 1500, "k": 14, "p": 0.5}
    ]
    
    for case in test_cases:
        ar_g = ar.watts_strogatz_graph(case["n"], case["k"], case["p"], create_using=ar.Graph)
        
        # Test 1: Vertex count should be exact
        assert len(ar_g) == case["n"]
        
        # Test 2: Edge count should be within 5% of expected
        expected_edges = case["n"] * case["k"] // 2
        assert abs(ar_g.size() - expected_edges) / expected_edges < 0.05

In [4]:
def test_barabasi_albert():
    """Test cases for Barabási-Albert graph generation with self-loop removal"""
    test_cases = [
        {"n": 100, "m": 2},
        {"n": 500, "m": 3},
        {"n": 1000, "m": 4},
        {"n": 2000, "m": 5},
        {"n": 5000, "m": 6},
        {"n": 200, "m": 3},
        {"n": 300, "m": 4},
        {"n": 400, "m": 5},
        {"n": 800, "m": 6},
        {"n": 1500, "m": 7}
    ]
    
    for case in test_cases:
        ar_g = ar.barabasi_albert_graph(case["n"], case["m"], create_using=ar.Graph)
        
        # Test 1: Vertex count should be exact
        assert len(ar_g) == case["n"]
        
        # Test 2: Edge count should be less than or equal to the theoretical maximum
        # Due to self-loop removal, actual edge count will be slightly lower
        max_possible_edges = case["m"] * (case["n"] - case["m"])
        assert ar_g.size() <= max_possible_edges
        
        # Test 3: Edge count should be reasonably close to theoretical maximum
        # Typically no more than 5% difference due to self-loop removal
        min_expected_edges = int(0.95 * max_possible_edges)
        assert ar_g.size() >= min_expected_edges
        
        # Test 4: Power law degree distribution (check slope of log-log plot)
        values, counts = ak.value_counts(ar_g.degree())
        
        # Take log2 of both arrays.
        values_log = np.log2(values.to_ndarray())
        counts_log = np.log2(counts.to_ndarray())
        
        # Calculate power law fit
        slope, _, _, _, _ = stats.linregress(values_log, counts_log)
        
        # Slope range adjusted for self-loop removal effects
        assert -2.0 < slope < -0.5  # Slightly wider range to account for variations

In [5]:
def test_gnp_random():
    """Test cases for G(n,p) random graph generation"""
    test_cases = [
        {"n": 100, "p": 0.1},
        {"n": 500, "p": 0.05},
        {"n": 1000, "p": 0.01},
        {"n": 2000, "p": 0.005},
        {"n": 5000, "p": 0.002},
        {"n": 200, "p": 0.15},
        {"n": 300, "p": 0.12},
        {"n": 400, "p": 0.08},
        {"n": 800, "p": 0.04},
        {"n": 1500, "p": 0.02}
    ]
    
    for case in test_cases:
        ar_g = ar.gnp_random_graph(case["n"], case["p"], create_using=ar.DiGraph)
        
        # Test 1: Vertex count should be less than or equal to n
        # Some vertices might have no edges and thus not appear in the graph
        assert len(ar_g) <= case["n"]
        
        # Test 2: Edge count should follow binomial distribution
        n_actual = len(ar_g)  # Use actual number of vertices for calculation
        expected_edges = n_actual * (n_actual-1) * case["p"]
        std_dev = np.sqrt(n_actual * (n_actual-1) * case["p"] * (1-case["p"]))
        
        # Check if edge count is within 3 standard deviations
        assert abs(ar_g.size() - expected_edges) < 3 * std_dev

In [6]:
def test_rmat():
    """Test cases for R-MAT graph generation"""
    test_cases = [
        {"scale": 8},
        {"scale": 9},
        {"scale": 10},
        {"scale": 11},
        {"scale": 12},
        {"scale": 13},
        {"scale": 14},
        {"scale": 15},
        {"scale": 16},
        {"scale": 17}
    ]
    
    for case in test_cases:
        ar_g = ar.rmat_graph(case["scale"], create_using=ar.Graph)
        
        # Test 1: Vertex count should be power of 2 or less
        assert len(ar_g) <= 2**case["scale"]
        
        # Test 2: Power law degree distribution (check slope of log-log plot)
        values, counts = ak.value_counts(ar_g.degree())
        
        # Take log2 of both arrays.
        values_log = np.log2(values.to_ndarray())
        counts_log = np.log2(counts.to_ndarray())
        
        # Calculate power law fit
        slope, _, _, _, _ = stats.linregress(values_log, counts_log)
        
        # Slope range adjusted for self-loop removal effects
        assert -2.0 < slope < -0.5  # Slightly wider range to account for variations

In [7]:
test_watts_strogatz()

In [8]:
test_barabasi_albert()

In [9]:
test_gnp_random()

In [10]:
test_rmat()